In [1]:
import numpy as np
from models import create_mlp_model

### Load iris data set just for test.

In [2]:
from sklearn.datasets import load_iris
from keras.utils import to_categorical

iris = load_iris()
X = iris['data']
y = iris['target']
# create the sparse target
y_sparse = to_categorical(y)

names = iris['target_names']
feature_names = iris['feature_names']

Using Theano backend.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 99, in init_dev
    **args)
  File "pygpu/gpuarray.pyx", line 658, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 587, in pygpu.gpuarray.pygpu_init
pygpu.gpuarray.GpuArrayException: b'Could not load "libnvrtc.so": libnvrtc.so: cannot open shared object file: No such file or directory'


In [3]:
y_sparse[:3]

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]], dtype=float32)

### Create the cross-validation.


In [4]:
from sklearn.model_selection import StratifiedKFold

n_folds = 10

skf = StratifiedKFold(n_splits=n_folds, random_state=17, shuffle=True)

print(skf)

StratifiedKFold(n_splits=10, random_state=17, shuffle=True)


In [5]:
# split the data set
train_test_indices = list(skf.split(X,y))
print('Number of folds %i; each fold has %i types os indices, train and test' %(len(train_test_indices),
                                                                               len(train_test_indices[0])))

Number of folds 10; each fold has 2 types os indices, train and test


### Train the model using the cross-validation and make $n$ initialization in order to avoid the local minima.

In [6]:
from tensorflow import keras

# try only a few initialization
n_inits = 3
# hyper-parameter for fit the model
n_epochs = 10
batch_size = 1024

# metrics and callbacks
metrics=[keras.metrics.sparse_top_k_categorical_accuracy, keras.metrics.categorical_accuracy]
callbacks = [keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=5,
        verbose=1)
]
# ======== hidden layer configuration ========================
number_of_hidden_layers = 1 # number of hidden layers in the model
hidden_layer_neurons = 5 # hiddem layer neurons
hidden_layer_act_function = 'relu' # the activation function

# ======== output layer configuration ========================
output_layer_neurons = 3 # there are 3 classes
output_layer_act_function = 'softmax' # output activate function

#======== compile model parameters ===========================
optimizer = 'adam' # there are some others but this is the mostly used
loss_function = 'categorical_crossentropy' # we have prepared the y making then sparse

In [15]:
from sklearn.preprocessing import StandardScaler

best_acc = 0.0

for idx, ifold in enumerate(train_test_indices):
    train_id, test_id = ifold[0], ifold[1]
    for init in range(n_inits):
        print('Training in the Fold: %i | Init: %i' %(idx+1, init+1))
        # create a scaler to prepare the data and fit using only the train data
        scl_factor = StandardScaler()
        scl_factor.fit(X[train_id])
        # transform all data
        X_norm = scl_factor.transform(X)
        
        # create the model
        model = create_mlp_model(input_dim=X_norm.shape[1], output_dim=output_layer_neurons,
                                 n_neurons=hidden_layer_neurons, n_layers=number_of_hidden_layers,
                                 hl_act_func=hidden_layer_act_function,
                                 ol_act_func=output_layer_act_function,
                                 loss_func=loss_function,
                                 optimizer=optimizer,
                                 metrics=metrics,
                                 name='model_fold{}_init{}'.format(idx+1, init+1)
                                )
        # fit he model using the data
        train_evo = model.fit(X_norm[train_id], y_sparse[train_id],
                             batch_size=batch_size, epochs=n_epochs,
                             callbacks=callbacks,
                             validation_data=(X_norm[test_id], y_sparse[test_id]),
                             verbose=0)
        # evaluate the model
        score = model.evaluate(X_norm[test_id], y_sparse[test_id])
    print('~~~~~~~~~~ End the training in Fold ~~~~~~~~~~')

Training in the Fold: 1 | Init: 1
15/15 [==============================]15/15 [==============================] - 0s 91us/step

Training in the Fold: 1 | Init: 2
15/15 [==============================]15/15 [==============================] - 0s 98us/step

Training in the Fold: 1 | Init: 3
15/15 [==============================]15/15 [==============================] - 0s 89us/step

~~~~~~~~~~ End the training in Fold ~~~~~~~~~~
Training in the Fold: 2 | Init: 1
15/15 [==============================]15/15 [==============================] - 0s 108us/step

Training in the Fold: 2 | Init: 2
15/15 [==============================]15/15 [==============================] - 0s 169us/step

Training in the Fold: 2 | Init: 3
15/15 [==============================]15/15 [==============================] - 0s 108us/step

~~~~~~~~~~ End the training in Fold ~~~~~~~~~~
Training in the Fold: 3 | Init: 1
15/15 [==============================]15/15 [==============================] - 0s 101us/step

Training in t

In [17]:
model.metrics_names

['loss', 'sparse_top_k_categorical_accuracy', 'categorical_accuracy']

In [19]:
train_evo.history['val_loss']

[1.5880674123764038,
 1.583601713180542,
 1.5788795948028564,
 1.5736991167068481,
 1.5688117742538452,
 1.5637754201889038,
 1.5587480068206787,
 1.5537278652191162,
 1.548530101776123,
 1.5433226823806763]

In [16]:
score

[1.5433226823806763, 1.0, 0.53333336114883423]